Full definition of a GPT Language Model, all of it in this single file.

References:
1) the official GPT-2 TensorFlow implementation released by OpenAI:
https://github.com/openai/gpt-2/blob/master/src/model.py
2) huggingface/transformers PyTorch implementation:
https://github.com/huggingface/transformers/blob/main/src/transformers/models/gpt2/modeling_gpt2.py

In [34]:
import math
import inspect
from dataclasses import dataclass

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch import Tensor
from typing import Optional, Tuple
from torch.utils.data import Dataset, DataLoader
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group

import os
import time
import pickle
from contextlib import nullcontext
from pathlib import Path
import fitz
import zipfile
import re

import numpy as np

# import requests
import tiktoken
from tqdm import tqdm
import random
import nltk

from transformers import GPT2Tokenizer
import random
from sklearn.model_selection import train_test_split

In [35]:
class LayerNorm(nn.Module):
    """ LayerNorm but with an optional bias. PyTorch doesn't support simply bias=False """

    def __init__(self,
                 ndim: int,
                 bias: Optional[bool]=True
                 ):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(ndim))
        self.bias = nn.Parameter(torch.zeros(ndim)) if bias else None

    def forward(self, input):
        return F.layer_norm(input, self.weight.shape, self.weight, self.bias, 1e-5)

In [36]:
def matts_scaled_dot_product_attention(query:     Tensor,
                                       key:       Tensor,
                                       value:     Tensor,
                                       mask:      Optional[Tensor]=None,
                                       dropout_p: Optional[float]=0.0,
                                       is_causal: Optional[bool]=False,
                                       scale:     Optional[float]=None
                                       ) -> Tensor:
    """
    Combine three tensors; query, key, and value; to generate an output tensor of scaled dot product attention.

    Parameters:
    - query (Tensor)              - shape (N x ... x L x E)
    - key (Tensor)                - shape (N x ... x S x E)
    - value (Tensor)              - shape (N x ... x S x Ev)
    - mask (optional Tensor)      - shape (N x ... x L x S)

            mask; shape must be broadcastable to the shape of attention weights.
                            Two types of masks are supported.
                                1) A boolean mask where a value of True indicates that the element should take part in attention.
                                2) A float mask of the same type as query, key, value that is added to the attention score.

    - dropout_p (float)           - Dropout probability; if greater than 0.0, dropout is applied

    Returns:
    - Attention output (Tensor)   - shape (N x ... x L x Ev)


    Shape legend:
    - N:    Batch size
    - ...:  Any number of other batch dimensions (optional)
    - S:    Source sequence length
    - L:    Target sequence length
    - E:    Embedding dimension of the query and key
    - Ev:   Embedding dimension of the value
    """
    L, S = query.size(-2), key.size(-2)

    # Calculate scaling factor ahead of time
    scale_factor = 1 / math.sqrt(query.size(-1)) if scale is None else scale

    # Pre-define attn_bias as zero-weighted tensor
    # this allows it to be included in the attn_weight
    # calculation regardless of being defined
    attn_bias = torch.zeros(L, S, dtype=query.dtype)

    if is_causal:
        temp_mask = torch.ones(L, S, dtype=torch.bool).tril(diagonal=0)
        attn_bias.masked_fill_(temp_mask.logical_not(), float("-inf"))
        attn_bias.to(query.dtype)

    if mask is not None:
        if mask.dtype == torch.bool:
            attn_bias.masked_fill_(mask.logical_not(), float("-inf"))
        else:
            attn_bias += mask

    # Compute attention weights
    attn_weight = query @ key.transpose(-2, -1) * scale_factor
    attn_weight += attn_bias

    # Apply softmax to the attention weights
    attn_weight = torch.softmax(attn_weight, dim=-1)

    # Apply dropout if specified
    if dropout_p > 0:
        attn_weight = torch.dropout(attn_weight, dropout_p, train=True)

    # Compute the final output
    return attn_weight @ value

In [37]:
class MultiheadAttention(nn.Module):

    def __init__(self,
                 num_heads: int,
                 num_embed: int,
                 bias:      Optional[bool]=True,
                 dropout_p: Optional[float]=0.0,
                 is_causal: Optional[bool]=False
                 ):
        super().__init__()
        assert num_embed % num_heads == 0, "Embedding dimension must be 0 modulo number of heads."

        self.num_heads = num_heads
        self.num_embed = num_embed
        self.bias      = bias
        self.dropout_p = dropout_p
        self.is_causal = is_causal

        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(num_embed, 3 * num_embed, bias=bias)
        # output projection
        self.c_proj = nn.Linear(num_embed, num_embed, bias=bias)
        # regularization
        self.attn_dropout = nn.Dropout(dropout_p)
        self.resid_dropout = nn.Dropout(dropout_p)

        # flash attention make GPU go brrrrr but support is only in PyTorch >= 2.0
        self.flash = hasattr(torch.nn.functional, 'scaled_dot_product_attention')

    def forward(self, x, attn_mask=None):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (embed_dim)

        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        q, k, v  = self.c_attn(x).split(self.num_embed, dim=2)
        k = k.view(B, T, self.num_heads, C // self.num_heads).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.num_heads, C // self.num_heads).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.num_heads, C // self.num_heads).transpose(1, 2) # (B, nh, T, hs)

        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        if self.flash:
            # efficient attention using Flash Attention CUDA kernels
            y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=attn_mask, dropout_p=self.dropout_p, is_causal=self.is_causal)
        else:
            # My version is essentially equivalent, but isn't optimised in the backend
            y = matts_scaled_dot_product_attention(q, k, v, attn_mask=attn_mask, dropout_p=self.dropout_p, is_causal=self.is_causal)

        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

        # output projection
        y = self.resid_dropout(self.c_proj(y))
        return y

In [38]:
class MLP(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.c_fc    = nn.Linear(config.n_embd, 4 * config.n_embd, bias=config.bias)
        self.gelu    = nn.GELU()
        self.c_proj  = nn.Linear(4 * config.n_embd, config.n_embd, bias=config.bias)
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        x = self.dropout(x)
        return x

In [39]:

class GPTDecoderBlock(nn.Module):
    """
    The decoder block described in:
     - Liu et al (2018) https://arxiv.org/pdf/1801.10198
     - Radford et al (2018) https://hayate-lab.com/wp-content/uploads/2023/05/43372bfa750340059ad87ac8e538c53b.pdf

      Consists of:
       - Masked Multi Self Attention layer
       - First Layer normalisation
       - Feed-forward network with two layers
       - Second Layer normalisation
    """

    def __init__(self,
                 num_heads: int,
                 num_embed: int,
                 bias:      Optional[bool]=True,
                 dropout_p: Optional[float]=0.0
                 ):
        """
        Inputs:
            input_dim (int)       - Dimensionality of the input
            num_heads (int)       - Number of heads to use in the attention block
            dim_feedforward (int) - Dimensionality of the hidden layer in the MLP
            dropout_p (float)     - Dropout probability to use in the dropout layers
        """
        super().__init__()

        self.num_heads = num_heads
        self.num_embed = num_embed
        self.bias      = bias
        self.dropout_p = dropout_p

        # Attention layer
        self.masked_self_attn = MultiheadAttention(num_heads, num_embed, bias, dropout_p, is_causal=True)

        # Two-layer MLP
        self.linear_net = nn.Sequential(
            nn.Linear(num_embed, 4*num_embed, bias=bias),
            nn.Dropout(dropout_p), # Extra dropout step inserted here to ensure dropout is included after every Linear layer
            nn.GELU(),             # GPT-2 applies GELU, deviating from ReLU in "Attention is all you need"
            nn.Linear(4*num_embed, num_embed, bias=bias)
        )

        # Layers to apply in between the main layers
        self.norm1 = LayerNorm(num_embed)
        self.norm2 = LayerNorm(num_embed)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self,
                x: Tensor,
                attn_mask: Optional[Tensor] = None
                )->Tensor:
        """
        Forward pass of the Transformer decoder block.

        Parameters
        ----------
        x : torch.Tensor
            The input tensor.
            Shape: [batch size, sequence length, embedding dim]
        attn_mask : torch.Tensor
            The mask to be applied to the attention scores.
            Shape: [batch size, 1, 1, sequence length]

        Returns
        -------
        torch.Tensor
            The output tensor of the Transformer encoder block.
            Shape: [batch size, sequence length, embedding dim]
        """

        # Self-attention part
        x = x + self.masked_self_attn(self.norm1(x),attn_mask)

        # MLP part
        x = x + self.linear_net(self.norm2(x))

        return x

In [40]:

class GPT(nn.Module):

    def __init__(self,
                 block_size: int,
                 vocab_size: int, # GPT-2 vocab_size of 50257, padded up to nearest multiple of 64 for efficiency
                 num_layer: int,
                 num_heads: int,
                 num_embed: int,
                 bias:      Optional[bool]=True,
                 dropout_p: Optional[float]=0.0
                 ):
        """
        A custom Transformer block, consisting of a stack of encoder and decoder layers.

        Parameters
        ----------
        num_decoder_layers : int
            The number of decoder layers in the Transformer block.
        input_dim : int
            The dimension of the input embeddings.
        num_heads : int
            The number of attention heads.
        dim_feedforward : int
            The inner size of the feed-forward networks in the encoder and decoder layers.
        dropout_p : float, optional, default=0.1
            The dropout rate.

        Attributes
        ----------
        encoder : torch.nn.ModuleList
            A list of TransformerEncoderBlock layers.
        decoder : torch.nn.ModuleList
            A list of TransformerDecoderBlock layers.
        """
        super(GPT, self).__init__()

        self.block_size = block_size
        self.vocab_size = vocab_size
        self.num_heads  = num_heads
        self.num_embed  = num_embed
        self.bias       = bias
        self.dropout_p  = dropout_p

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(self.vocab_size, self.num_embed),
            wpe = nn.Embedding(self.block_size, self.num_embed),
            drop = nn.Dropout(self.dropout_p),
            h = nn.ModuleList([GPTDecoderBlock(num_heads,num_embed,bias,dropout_p) for _ in range(num_layer)]),
            ln_f = LayerNorm(self.num_embed, bias=self.bias),
        ))
        self.lm_head = nn.Linear(num_embed, vocab_size, bias=False)
        # with weight tying when using torch.compile() some warnings get generated:
        # "UserWarning: functional_call was passed multiple values for tied weights.
        # This behavior is deprecated and will be an error in future versions"
        # not 100% sure what this is, so far seems to be harmless. TODO investigate
        self.transformer.wte.weight = self.lm_head.weight # https://paperswithcode.com/method/weight-tying

        # init all weights
        self.apply(self._init_weights)
        # apply special scaled init to the residual projections, per GPT-2 paper
        for pn, p in self.named_parameters():
            if pn.endswith('c_proj.weight'):
                torch.nn.init.normal_(p, mean=0.0, std=0.02/math.sqrt(2 * num_layer))

        # report number of parameters
        print("number of parameters: %.2fM" % (self.get_num_params()/1e6,))

    def get_num_params(self, non_embedding=True):
        """
        Return the number of parameters in the model.
        For non-embedding count (default), the position embeddings get subtracted.
        The token embeddings would too, except due to the parameter sharing these
        params are actually used as weights in the final layer, so we include them.
        """
        n_params = sum(p.numel() for p in self.parameters())
        if non_embedding:
            n_params -= self.transformer.wpe.weight.numel()
        return n_params

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        device = idx.device
        b, t = idx.size()
        assert t <= self.block_size, f"Cannot forward sequence of length {t}, block size is only {self.block_size}"
        pos = torch.arange(0, t, dtype=torch.long, device=device) # shape (t)

        # forward the GPT model itself
        tok_emb = self.transformer.wte(idx) # token embeddings of shape (b, t, n_embd)
        pos_emb = self.transformer.wpe(pos) # position embeddings of shape (t, n_embd)
        x = self.transformer.drop(tok_emb + pos_emb)
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)

        if targets is not None:
            # if we are given some desired targets also calculate the loss
            logits = self.lm_head(x)
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
        else:
            # inference-time mini-optimization: only forward the lm_head on the very last position
            logits = self.lm_head(x[:, [-1], :]) # note: using list [-1] to preserve the time dim
            loss = None

        return logits, loss

    def configure_optimizers(self, weight_decay, learning_rate, betas, device_type):
        # start with all of the candidate parameters
        param_dict = {pn: p for pn, p in self.named_parameters()}
        # filter out those that do not require grad
        param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}
        # create optim groups. Any parameters that is 2D will be weight decayed, otherwise no.
        # i.e. all weight tensors in matmuls + embeddings decay, all biases and layernorms don't.
        decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
        nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
        optim_groups = [
            {'params': decay_params, 'weight_decay': weight_decay},
            {'params': nodecay_params, 'weight_decay': 0.0}
        ]
        num_decay_params = sum(p.numel() for p in decay_params)
        num_nodecay_params = sum(p.numel() for p in nodecay_params)
        print(f"num decayed parameter tensors: {len(decay_params)}, with {num_decay_params:,} parameters")
        print(f"num non-decayed parameter tensors: {len(nodecay_params)}, with {num_nodecay_params:,} parameters")
        # Create AdamW optimizer and use the fused version if it is available
        fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
        use_fused = fused_available and device_type == 'cuda'
        extra_args = dict(fused=True) if use_fused else dict()
        optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=betas, **extra_args)
        print(f"using fused AdamW: {use_fused}")

        return optimizer

    def estimate_mfu(self, fwdbwd_per_iter, dt):
        """ estimate model flops utilization (MFU) in units of A100 bfloat16 peak FLOPS """
        # first estimate the number of flops we do per iteration.
        # see PaLM paper Appendix B as ref: https://arxiv.org/abs/2204.02311
        N = self.get_num_params()
        L, H, Q, T = self.num_layer, self.num_heads, self.num_embed//self.num_heads, self.block_size
        flops_per_token = 6*N + 12*L*H*Q*T
        flops_per_fwdbwd = flops_per_token * T
        flops_per_iter = flops_per_fwdbwd * fwdbwd_per_iter
        # express our flops throughput as ratio of A100 bfloat16 peak flops
        flops_achieved = flops_per_iter * (1.0/dt) # per second
        flops_promised = 312e12 # A100 GPU bfloat16 peak flops is 312 TFLOPS
        mfu = flops_achieved / flops_promised
        return mfu

    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        """
        Take a conditioning sequence of indices idx (LongTensor of shape (b,t)) and complete
        the sequence max_new_tokens times, feeding the predictions back into the model each time.
        Most likely you'll want to make sure to be in model.eval() mode of operation for this.
        """
        for _ in range(max_new_tokens):
            # if the sequence context is growing too long we must crop it at block_size
            idx_cond = idx if idx.size(1) <= self.block_size else idx[:, -self.block_size:]
            # forward the model to get the logits for the index in the sequence
            logits, _ = self(idx_cond)
            # pluck the logits at the final step and scale by desired temperature
            logits = logits[:, -1, :] / temperature
            # optionally crop the logits to only the top k options
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('Inf')
            # apply softmax to convert logits to (normalized) probabilities
            probs = F.softmax(logits, dim=-1)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)
            # append sampled index to the running sequence and continue
            idx = torch.cat((idx, idx_next), dim=1)

        return idx

## Data Preparation
My first model will be trained on shakespeare_and_apra.txt. 
</br>A small 2MB toy dataset, which consists of:
1) 40k lines of text containing the collected works of shakespere, downloaded from https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
2) 40k lines of text extracted from PDFs sourced from the Australian Prudential Regulation Authority (APRA) bank risk reporting regulations 

Theoretically I could use a larger dataset to build a better map of language relationships. However, I lack the computing power to do that approach justice.
</br>So instead I chose an approach that I found funny. As I fully expect the resulting model to generate an amusing combination of shakesperian language and dry descriptions of bank regulations.

This dataset is processed as follows:
1) Apply the same byte encoding as GPT-2
2) Split into training and validation datasets

Define paths and create folders

In [41]:
root_path=Path(os.getcwd())
data_path=root_path.joinpath('data')
corpus_path=data_path.joinpath('corpus')
pdf_path=data_path.joinpath('apra_pdfs')

Path(data_path).mkdir(parents=True, exist_ok=True)
Path(corpus_path).mkdir(parents=True, exist_ok=True)
Path(pdf_path).mkdir(parents=True, exist_ok=True)

#### Split shakespear file into 20 even parts
This makes it a bit easier to fairly corpus into train and test portions with APRA guidelines. As we want an even mix of Shakespeare text and banking regulations in both corpus.

In [42]:
def split_file(input_filename: str,
               source_path: Path,
               target_path: Path,
               num_chunks: Optional[int]=20
               ):
    
    # Read all the lines from the input file
    with open(source_path.joinpath(input_filename), 'r') as file:
        lines = file.readlines()

    # Calculate the number of lines per chunk
    total_lines = len(lines)
    lines_per_chunk = total_lines // num_chunks
    remainder = total_lines % num_chunks  # For uneven splits

    for i in range(num_chunks):
        # Calculate start and end indices for each chunk
        start = i * lines_per_chunk + min(i, remainder)
        end = start + lines_per_chunk + (1 if i < remainder else 0)

        # Generate a chunk filename
        chunk_filename = target_path.joinpath(f"{input_filename[:-4]}_{i+1:02}.txt")

        # Write the current chunk to a new file
        with open(chunk_filename, 'w') as chunk_file:
            chunk_file.writelines(lines[start:end])

        print(f"Written: {chunk_filename}")

In [43]:
split_file('shakespear.txt',data_path,corpus_path)

Written: d:\repos\Transformers_from_scratch\data\corpus\shakespear_01.txt
Written: d:\repos\Transformers_from_scratch\data\corpus\shakespear_02.txt
Written: d:\repos\Transformers_from_scratch\data\corpus\shakespear_03.txt
Written: d:\repos\Transformers_from_scratch\data\corpus\shakespear_04.txt
Written: d:\repos\Transformers_from_scratch\data\corpus\shakespear_05.txt
Written: d:\repos\Transformers_from_scratch\data\corpus\shakespear_06.txt
Written: d:\repos\Transformers_from_scratch\data\corpus\shakespear_07.txt
Written: d:\repos\Transformers_from_scratch\data\corpus\shakespear_08.txt
Written: d:\repos\Transformers_from_scratch\data\corpus\shakespear_09.txt
Written: d:\repos\Transformers_from_scratch\data\corpus\shakespear_10.txt
Written: d:\repos\Transformers_from_scratch\data\corpus\shakespear_11.txt
Written: d:\repos\Transformers_from_scratch\data\corpus\shakespear_12.txt
Written: d:\repos\Transformers_from_scratch\data\corpus\shakespear_13.txt
Written: d:\repos\Transformers_from_sc

#### Extract APRA_Guidelines.zip and convert PDFs into text files

In [44]:
with zipfile.ZipFile(data_path.joinpath('APRA_Guidelines.zip'), 'r') as zip_ref:
    zip_ref.extractall(pdf_path)
    print(f"Extracted 'APRA_Guidelines.zip' to '{pdf_path}'.")

def pdf_to_text(pdf_path: Path, txt_path: Path):
    # Open the PDF
    pdf_document = fitz.open(pdf_path)

    # Create a text file to store the extracted text
    with open(txt_path, "w", encoding="utf-8") as text_file:
        for page_number in range(len(pdf_document)):
            page = pdf_document.load_page(page_number)
            text = page.get_text()
            text_file.write(text)

    # Close the PDF
    pdf_document.close()

def convert_pdfs_in_folder(source_path: Path, target_path: Path):
    for file in os.listdir(source_path):
        if file.endswith('.pdf'):
            pdf_to_text(source_path.joinpath(file), target_path.joinpath(re.sub('.pdf','.txt',file)))
            print(f'Created {target_path.joinpath(re.sub('.pdf','.txt',file))}')


Extracted 'APRA_Guidelines.zip' to 'd:\repos\Transformers_from_scratch\data\apra_pdfs'.


In [45]:
convert_pdfs_in_folder(pdf_path,corpus_path)

Created d:\repos\Transformers_from_scratch\data\corpus\3PS 222 Intra-group Transactions and Exposures.txt
Created d:\repos\Transformers_from_scratch\data\corpus\APG 223 Residential Mortgage Lending.txt
Created d:\repos\Transformers_from_scratch\data\corpus\APS 110 Capital Adequacy.txt
Created d:\repos\Transformers_from_scratch\data\corpus\APS 112 Capital Adequacy Standardised Approach.txt
Created d:\repos\Transformers_from_scratch\data\corpus\APS 113 Capital Adequacy Internal Ratings-based.txt
Created d:\repos\Transformers_from_scratch\data\corpus\APS 180 Capital Adequacy Counterparty Credit Risk.txt
Created d:\repos\Transformers_from_scratch\data\corpus\APS 210 Liquidity.txt
Created d:\repos\Transformers_from_scratch\data\corpus\APS 220 Credit Risk Management.txt
Created d:\repos\Transformers_from_scratch\data\corpus\APS 222 Associations with Related Entities.txt
Created d:\repos\Transformers_from_scratch\data\corpus\ARS 118 Off-balance Sheet Business.txt
Created d:\repos\Transformers

#### For interest, combine all files and sample together to get a sense of the text contained therein

In [46]:
def append_from_folder(source_path: Path, output_file: Path):
    # Open the output file in write mode
    with open(source_path.joinpath(output_file), "w", encoding="utf-8") as outfile:
        # Loop through all files in the folder
        for filename in os.listdir(source_path):
            # Check if the file is a .txt file
            if filename.endswith(".txt"):
                file_path = os.path.join(source_path, filename)
                # Open the text file and read its contents
                with open(file_path, "r", encoding="utf-8") as infile:
                    content = infile.read()
                    # Write the content of each text file into the output file
                    outfile.write(content)
                    # Optionally, add a separator between files
                    outfile.write("\n\n--- End of file: {} ---\n\n".format(filename))

    print(f"All text files in {source_path} have been appended to {output_file}.")


In [47]:
append_from_folder(corpus_path,data_path.joinpath('shakespeare_and_apra.txt'))

All text files in d:\repos\Transformers_from_scratch\data\corpus have been appended to d:\repos\Transformers_from_scratch\data\shakespeare_and_apra.txt.


In [48]:
def get_random_line(filepath: str) -> str:
    file_size = os.path.getsize(filepath)
    with open(filepath, 'rb') as f:
        while True:
            pos = random.randint(0, file_size)
            if not pos:  # the first line is chosen
                return pos,f.readline().decode()  # return str
            f.seek(pos)  # seek to random position
            f.readline()  # skip possibly incomplete line
            line = f.readline()  # read next (full) line
            if line:
                return pos,line.decode()
            # else: line is empty -> EOF -> try another position in next iteration

In [49]:
print('===================================================')
print('Sampling random lines from shakespeare_and_apra.txt')
print('===================================================')
for i in range(10):
    pos,line=get_random_line(data_path.joinpath('shakespeare_and_apra.txt'))
    print(f'Row {pos}:',line)

Sampling random lines from shakespeare_and_apra.txt
Row 466956: collateral or a change in the number of transactions in the netting set. For 

Row 104902: 33. 

Row 1153556: 7  

Row 1027150: 13

Row 936213: Authorised Version F2019L00828 registered 17/06/2019

Row 585264: supervisory correlation parameter and the supervisory volatility are as given in 

Row 429895: If there is a take-or-pay or 

Row 2287433: 

Row 364647: models and other mechanical methods used to assign borrower or facility grades 

Row 1769668: They'll be in scarlet straight at any news.



### Prepare raw text corpus for training

The following was generated from ChatGPT model 4o using the following prompt:
"""
I have a set of 39 text files that will be used as a corpus into a transformer NLP model using Pytorch. Can you please provide a guide to do the following:
1) Randomly assign each document into a train and test corpus for validation, using an 80:20 split
2) Apply the same byte encoding methodology that was applied to GPT2
3) Create a dataset and data loader using Pytorch syntax
"""

</br> Chat link: https://chatgpt.com/c/6709de73-31a0-8000-b340-1b6b0c5705cc

#### 1. Randomly Assign Each Document to Train and Test Corpus (80:20 Split)

In [50]:
import random
from sklearn.model_selection import train_test_split

def split_files_into_train_test(file_list, test_size=0.2, random_seed=42):
    # Set the random seed for reproducibility
    random.seed(random_seed)
    
    # Split the list into training and testing sets (80% train, 20% test)
    train_files, val_files = train_test_split(file_list, test_size=test_size, random_state=random_seed)
    
    return train_files, val_files
file_list = [corpus_path.joinpath(file) for file in os.listdir(corpus_path) if file.endswith('.txt')]
train_files, val_files = split_files_into_train_test(file_list)

In [51]:
print('===================================================')
print("Documents in training corpus")
print('===================================================')
train_files

Documents in training corpus


[WindowsPath('d:/repos/Transformers_from_scratch/data/corpus/shakespear_06.txt'),
 WindowsPath('d:/repos/Transformers_from_scratch/data/corpus/CPS 230 Operational Risk Management.txt'),
 WindowsPath('d:/repos/Transformers_from_scratch/data/corpus/CPS 232 Business Continuity.txt'),
 WindowsPath('d:/repos/Transformers_from_scratch/data/corpus/APS 222 Associations with Related Entities.txt'),
 WindowsPath('d:/repos/Transformers_from_scratch/data/corpus/CPS 231 Outsourcing.txt'),
 WindowsPath('d:/repos/Transformers_from_scratch/data/corpus/CPG 229 Climate Change Financial Risks.txt'),
 WindowsPath('d:/repos/Transformers_from_scratch/data/corpus/shakespear_01.txt'),
 WindowsPath('d:/repos/Transformers_from_scratch/data/corpus/ARS 118 Off-balance Sheet Business.txt'),
 WindowsPath('d:/repos/Transformers_from_scratch/data/corpus/shakespear_14.txt'),
 WindowsPath('d:/repos/Transformers_from_scratch/data/corpus/3PS 222 Intra-group Transactions and Exposures.txt'),
 WindowsPath('d:/repos/Transfo

In [52]:
print('===================================================')
print("Documents in validation corpus")
print('===================================================')
val_files

Documents in validation corpus


[WindowsPath('d:/repos/Transformers_from_scratch/data/corpus/shakespear_15.txt'),
 WindowsPath('d:/repos/Transformers_from_scratch/data/corpus/shakespear_18.txt'),
 WindowsPath('d:/repos/Transformers_from_scratch/data/corpus/APS 113 Capital Adequacy Internal Ratings-based.txt'),
 WindowsPath('d:/repos/Transformers_from_scratch/data/corpus/CPG-235-Managing-Data-Risk.txt'),
 WindowsPath('d:/repos/Transformers_from_scratch/data/corpus/shakespear_12.txt'),
 WindowsPath('d:/repos/Transformers_from_scratch/data/corpus/shakespear_08.txt'),
 WindowsPath('d:/repos/Transformers_from_scratch/data/corpus/APS 210 Liquidity.txt'),
 WindowsPath('d:/repos/Transformers_from_scratch/data/corpus/shakespear_09.txt')]

In [53]:
def concatenate_files_to_output(file_list, output_file):
    """
    Concatenate text from multiple files and write to a single output file.
    
    Parameters:
    - file_list: List of paths to the input text files.
    - output_file: Path to the output text file where combined text will be saved.
    """
    with open(output_file, 'w', encoding='utf-8') as outfile:
        for file_path in file_list:
            # Ensure the file exists
            if os.path.exists(file_path):
                with open(file_path, 'r', encoding='utf-8') as infile:
                    # Read the contents of the file and write to output file
                    outfile.write(infile.read() + "\n")  # Add a newline to separate contents
            else:
                print(f"File not found: {file_path}")

train_text_path = data_path.joinpath('train.txt')
concatenate_files_to_output(train_files, train_text_path)
print(f"Combined text has been written to: {train_text_path}")

val_text_path = data_path.joinpath('val.txt')
concatenate_files_to_output(val_files, val_text_path)
print(f"Combined text has been written to: {val_text_path}")

Combined text has been written to: d:\repos\Transformers_from_scratch\data\train.txt
Combined text has been written to: d:\repos\Transformers_from_scratch\data\val.txt


In [54]:
enc = tiktoken.get_encoding("gpt2")

def apply_gpt2_byte_encoding(data_path, source_file, target_file):
    with open(data_path.joinpath(source_file), 'r', encoding='utf-8') as f:
        text = f.read()

        # Some low level text cleaning
        text = re.sub(r'\n\s*\n', '\n', text.strip())
        text = re.sub(r'[^\x00-\x7F]+', '', text)

        # encode with tiktoken gpt2 bpe
        encoded_text = enc.encode_ordinary(text)
        print(f"{source_file} has {len(encoded_text):,} tokens")
        encoded_text = np.array(encoded_text, dtype=np.uint16)
        encoded_text.tofile(data_path.joinpath(target_file))
        vocab_size = len(encoded_text)

train_vocab_size=apply_gpt2_byte_encoding(data_path,'train.txt','train.bin')
val_vocab_size=apply_gpt2_byte_encoding(data_path,'val.txt','val.bin')

train.txt has 457,046 tokens
val.txt has 161,208 tokens


In [55]:
import nltk
from transformers import GPT2Tokenizer

# Download the Punkt sentence tokenizer if you haven't already
nltk.download('punkt')

def preprocess_text(text):
    text = re.sub(r'\n\s*\n', '\n', text.strip())
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    return text

def chunk_sentences(sentences, tokenizer, max_length=1024):
    """
        Function to force text to be chunks that are 1024 or less. 
        Whole sentences are preserved, and padding inserted to make up to the text length.
    """
    current_chunk = []
    current_length = 0
    chunks = []

    long_sentence_cnt=0
    long_chunk_cnt=0

    # Iterate over sentences, tokenizing them, and grouping them into chunks
    for sentence in sentences:
        # Tokenize the sentence
        sentence_tokens = tokenizer.encode(sentence)

        if len(sentence_tokens) > max_length:
            long_sentence_cnt+=1

        # If adding this sentence exceeds the max length, save the current chunk and start a new one
        if current_length + len(sentence_tokens) > max_length:
            chunks.append(current_chunk)
            current_chunk = []
            current_length = 0

        # Add the sentence to the current chunk
        current_chunk.extend(sentence_tokens)
        current_length += len(sentence_tokens)

        if current_length > max_length:
            long_chunk_cnt+=1
            
    # Add the last chunk
    if current_chunk:
        chunks.append(current_chunk)

    if long_sentence_cnt>0:
        print('Num long sentences:',long_sentence_cnt)
    if long_chunk_cnt>0:
        print('Num long chunks:',long_sentence_cnt)
    return chunks

def apply_gpt2_encoding(files, max_length=1024, pad_token_id=50256):
    """
        Convert files in corpus into byte-encoded datasets

        Using the OpenAI tokeniser instead of the Hugging face version to avoid the following error:

    """
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    # tokenizer = tiktoken.get_encoding("gpt2")

    # Tokenize each file and convert to byte-level BPE
    all_encoded_chunks = []
    all_attention_masks = []

    for file in files:
        with open(file, 'r', encoding='utf-8') as f:
            text = f.read()

            # Preprocess and split into sentences
            text = preprocess_text(text)
            
            # Split into sentence-sized chunks
            sentences = nltk.sent_tokenize(text)
            encoded_chunks = chunk_sentences(sentences, tokenizer)

        for chunk in encoded_chunks:
            # Apply padding to ensure all chunks are of the same length
            padding_length = max_length - len(chunk)
            attention_mask = [1] * len(chunk) + [0] * padding_length
            padded_chunk = chunk + [pad_token_id] * padding_length
            
            all_encoded_chunks.append(padded_chunk)
            all_attention_masks.append(attention_mask)

    return all_encoded_chunks, all_attention_masks

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\crowl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [56]:
class TextDataset(Dataset):
    """
        Dataset constructor to feed into a Pytorch dataloader with both input data chunk 
        and attention mask for padded characters.
    """
    def __init__(self, encoded_chunks, attention_masks):
        self.encoded_chunks = encoded_chunks
        self.attention_masks = attention_masks

    def __len__(self):
        return len(self.encoded_chunks)

    def __getitem__(self, idx):

        return {
            'input_ids': torch.tensor(self.encoded_chunks[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_masks[idx], dtype=torch.long)
        }

In [ ]:
class GPTConfig:
    def __init__(self, vocab_size, **kwargs):
        self.vocab_size = vocab_size
        for key, value in kwargs.items():
            setattr(self, key, value)

class CustomConfig(GPTConfig):
    n_layer = 8
    n_head = 8
    n_embd = 256
    embd_pdrop = 0.1
    resid_pdrop = 0.1
    attn_pdrop = 0.1
    dropout = 0.1
    compile = True
    device = 'cuda'
    num_workers = 0
    max_iters = 2e4
    batch_size = 4
    block_size = 64
    learning_rate = 6e-4
    betas = (0.9, 0.95)
    weight_decay = 1e-1
    grad_norm_clip = 1.0

vocab_size = len(train_ids)
config = CustomConfig(vocab_size=vocab_size)

In [ ]:
# read data from .bin
data_dir = os.path.join('data', 'tinyshakespeare')
train_data = np.memmap(os.path.join(data_dir, 'train.bin'), dtype=np.uint16, mode='r')
val_data = np.memmap(os.path.join(data_dir, 'val.bin'), dtype=np.uint16, mode='r')

class ShakespeareDataset(Dataset):
    def __init__(self, split, block_size=128, device_type='cuda'):
        assert split in {'train', 'test'}
        self.split = split
        self.block_size = block_size
        self.device_type = device_type
        self.data = train_data if split == 'train' else val_data
    
    def __len__(self):
        return len(self.data) - self.block_size

    def __getitem__(self, idx):
        x = torch.from_numpy(self.data[idx : idx + self.block_size].astype(np.int64))
        y = torch.from_numpy(self.data[idx + 1 : idx + 1 + self.block_size].astype(np.int64)) 

        if self.device_type == 'cuda':
            # pin arrays x,y, which allows us to move them to GPU asynchronously (non_blocking=True)
            x, y = x.pin_memory().to('cuda', non_blocking=True), y.pin_memory().to('cuda', non_blocking=True)
        else:
            x, y = x.to('cpu'), y.to('cpu')
        return x, y

# create dataset and dataloader
train_dataset = ShakespeareDataset('train', config.block_size, config.device)
train_loader = DataLoader(train_dataset, batch_size=config.batch_size, num_workers=config.num_workers, drop_last=False)
test_dataset = ShakespeareDataset('test', config.block_size, config.device)
test_loader = DataLoader(test_dataset, batch_size=config.batch_size, num_workers=config.num_workers, drop_last=False)

In [57]:
# Prepare the dataset and dataloader
def create_dataloader(texts, batch_size=8, max_length=1024, pad_token_id=50256):
    """
        Create a dataloader from input list of files.
    """
    # Apply encoding and chunking
    encoded_chunks, attention_masks = apply_gpt2_encoding(texts, max_length, pad_token_id)

    # Create a dataset from the encoded data
    dataset = TextDataset(encoded_chunks, attention_masks)

    # Create a DataLoader for batching
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    return dataloader

In [58]:
train_loader = create_dataloader(train_files)
val_loader = create_dataloader(val_files)

c:\ProgramData\anaconda3\envs\torch-gpu\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Train model functions
Basic model training function, the following hyperparameters can be optimised:
 * learning_rate - The % rate of learning 
 * optimiser - Choice of optimisation algorithm, ADAM is default
 * loss_function - How loss is calculated for a given weight set (the optimisation algorithm attempts to minimise loss)
 * num_epochs - How many times the algorithmn will iterate through the dataset. 
 * patience - Early stopping algorithmn, How many ephochs to allow the algorithmn to iterate before reverting to the last best weight set. No early stopping if this option is not set

In [59]:
torch.manual_seed(1337)
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cuda' # for later use in torch.autocast

# note: float16 data type will automatically use a GradScaler
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32', 'bfloat16', or 'float16', the latter will auto implement a GradScaler
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

In [60]:
def get_lr(iter: int,
           learning_rate: float,
           lr_decay_iters: int, # should be ~= max_iters per Chinchilla
           warmup_iters: Optional[int]=2000, # how many steps to warm up for
           ):
    """
        learning rate decay scheduler (cosine with warmup)
    """
    
    # minimum learning rate, should be ~= learning_rate/10 per Chinchilla
    min_learning_rate=learning_rate/10

    # 1) linear warmup for warmup_iters steps
    if iter < warmup_iters:
        return learning_rate * iter / warmup_iters
    # 2) if iter > lr_decay_iters, return min learning rate
    if iter > lr_decay_iters:
        return min_learning_rate
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (iter - warmup_iters) / (lr_decay_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # coeff ranges 0..1
    return min_learning_rate + coeff * (learning_rate - min_learning_rate)

In [61]:
def get_batch(split, block_size, batch_size, device):
    """
        A simplified dataloader lifted from the NanoGPT repo.

        I originally attempted to create my own but found this one manages entire documents.
        So I don't have to worry as much about padding/masking block sizes.
    """
    # We recreate np.memmap every batch to avoid a memory leak, as per
    # https://stackoverflow.com/questions/45132940/numpy-memmap-memory-usage-want-to-iterate-once/61472122#61472122
    if split == 'train':
        data = np.memmap(data_path.joinpath('train.bin'), dtype=np.uint16, mode='r')
    else:
        data = np.memmap(data_path.joinpath('val.bin'), dtype=np.uint16, mode='r')
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([torch.from_numpy((data[i:i+block_size]).astype(np.int64)) for i in ix])
    y = torch.stack([torch.from_numpy((data[i+1:i+1+block_size]).astype(np.int64)) for i in ix])
    if device_type == 'cuda':
        # pin arrays x,y, which allows us to move them to GPU asynchronously (non_blocking=True)
        x, y = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(device, non_blocking=True)
    else:
        x, y = x.to(device), y.to(device)
    return x, y

In [62]:
# attempt to derive vocab_size from the dataset
meta_path = data_path.joinpath('meta.pkl')
meta_vocab_size = None
if os.path.exists(meta_path):
    with open(meta_path, 'rb') as f:
        meta = pickle.load(f)
    meta_vocab_size = meta['vocab_size']
    print(f"found vocab_size = {meta_vocab_size} (inside {meta_path})")

found vocab_size = 211 (inside d:\repos\Transformers_from_scratch\data\meta.pkl)


In [63]:
# helps estimate an arbitrarily accurate loss over either split using many batches
# @torch.no_grad()
# def estimate_loss(model, eval_iters, block_size, batch_size, device):
#     out = {}
#     model.eval()
#     for split in ['train', 'val']:
#         losses = torch.zeros(eval_iters)
#         for k in range(eval_iters):
#             X, Y = get_batch(split, block_size, batch_size, device)
#             with ctx:
#                 logits, loss = model(X, Y)
#             losses[k] = loss.item()
#         out[split] = losses.mean()
#     model.train()
#     return out

In [ ]:
# helps estimate an arbitrarily accurate loss over either split using many batches
@torch.no_grad()
def estimate_loss(model, eval_iters):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            with ctx:
                logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [68]:
results_path=root_path.joinpath('results')

# Function to perform forward pass
def model_forward(model, input_ids, attention_mask=None):
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    hidden_state = outputs[0]
    pooled_output = hidden_state[:, 0]
    logits = model.classifier(pooled_output)
    return logits


def train_model(model,

                train_loader,
                val_loader,

                # I/O
                eval_interval=100,
                log_interval=10,
                eval_iters = 200,
                always_save_checkpoint=False,
                out_dir=results_path,
                device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
                running_mfu=-1.0,  

                # data
                gradient_accumulation_steps=1,
                batch_size=64,
                block_size=1024,

                # adamw optimizer
                learning_rate=6e-4, # max learning rate
                weight_decay=1e-1,
                betas=(0.9,0.95),
                grad_clip=1.0, # clip gradients at this value, or disable if == 0.0

                # learning rate decay settings
                decay_lr=True, # whether to decay the learning rate
                num_epochs=60000,
                warmup_iters=2000, # how many steps to warm up for
                ):
    
    model.to(device)
    # model = torch.compile(model)

    scaler = torch.amp.GradScaler('cuda',enabled=(dtype == 'float16'))

    optimizer = model.configure_optimizers(weight_decay, learning_rate, betas, device)
    
    training_logs = []  # List to store training logs

    t0 = time.time()
    for epoch in range(num_epochs):

        train_progress_bar = tqdm(train_loader, desc=f"Training (Epoch {epoch+1}/{num_epochs})", leave=False)
        for batch in train_progress_bar:
            
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            
            logits = model_forward(model, input_ids, attention_mask=None)

        # determine and set the learning rate for this iteration
        epoch_lr = get_lr(epoch, learning_rate, num_epochs, warmup_iters)
        for param_group in optimizer.param_groups:
            param_group['lr'] = epoch_lr

        # evaluate the loss on train/val sets and write checkpoints
        if epoch % eval_interval == 0:
            losses = estimate_loss(model, eval_iters, block_size, batch_size, device)
            print(f"step {epoch}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
            training_logs.append({"Epoch": epoch,
                                  "train/loss": losses['train'],
                                  "val/loss": losses['val'],
                                  "lr": epoch_lr,
                                  "mfu": running_mfu*100, # convert to percentage
                                  })
            if losses['val'] < best_val_loss or always_save_checkpoint:
                best_val_loss = losses['val']
                if epoch > 0:
                    checkpoint = {
                        'model': model.state_dict(),
                        'optimizer': optimizer.state_dict(),
                        'iter_num': epoch,
                        'best_val_loss': best_val_loss
                    }
                    print(f"saving checkpoint to {out_dir}")
                    torch.save(checkpoint, os.path.join(out_dir, 'ckpt.pt'))

        # forward backward update, with optional gradient accumulation to simulate larger batch size
        # and using the GradScaler if data type is float16
        for micro_step in range(gradient_accumulation_steps):   
            with ctx:
                logits, loss = model(X, Y)
                loss = loss / gradient_accumulation_steps # scale the loss to account for gradient accumulation
            # immediately async prefetch next batch while model is doing the forward pass on the GPU
            X, Y = get_batch('train')
            # backward pass, with gradient scaling if training in fp16
            scaler.scale(loss).backward()
        # clip the gradient
        if grad_clip != 0.0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)

        # step the optimizer and scaler if training in fp16
        scaler.step(optimizer)
        scaler.update()
        # flush the gradients as soon as we can, no need for this memory anymore
        optimizer.zero_grad(set_to_none=True)

        # timing and logging
        t1 = time.time()
        dt = t1 - t0
        t0 = t1

        if epoch % log_interval == 0:
            # get loss as float. note: this is a CPU-GPU sync point
            # scale up to undo the division above, approximating the true total loss (exact would have been a sum)
            lossf = loss.item() * gradient_accumulation_steps
            if epoch >= 5: # let the training loop settle a bit
                mfu = model.estimate_mfu(batch_size * gradient_accumulation_steps, dt)
                running_mfu = mfu if running_mfu == -1.0 else 0.9*running_mfu + 0.1*mfu
            pbar.set_postfix(f"Epoch {epoch}: loss {lossf:.4f}, time {dt*1000:.2f}ms, mfu {running_mfu*100:.2f}%")
            pbar.update(1)

    # Close the progress bar if it was opened
    pbar.close()

    # Garbage collection on the GPU cache to avoid keeping unnecessary images/weights stored in memory
    torch.cuda.empty_cache()

    return model, training_logs

In [28]:
torch.cuda.empty_cache()

In [66]:
mattGPT_mini=GPT(block_size=1024,
            vocab_size=6400, # GPT-2 vocab_size of 50257, padded up to nearest multiple of 64 for efficiency
            num_layer=2,
            num_heads=2,
            num_embed=64,
            bias=True, # True: bias in Linears and LayerNorms, like GPT-2. False: a bit better and faster
            dropout_p=0.2
            )

number of parameters: 0.51M


In [70]:
mattGPT_mini_trained, mattGPT_mini_logs = train_model(

                mattGPT_mini,
                train_loader,
                val_loader,

                # I/O
                eval_interval=100,
                log_interval=10,
                eval_iters = 200,
                always_save_checkpoint=False,
                out_dir=results_path,
                device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
                running_mfu=-1.0,  

                # data
                gradient_accumulation_steps=1,
                batch_size=12,
                block_size=1024,

                # adamw optimizer
                learning_rate=6e-4, # max learning rate
                weight_decay=1e-1,
                betas=(0.9,0.95),
                grad_clip=1.0, # clip gradients at this value, or disable if == 0.0

                # learning rate decay settings
                decay_lr=True, # whether to decay the learning rate
                num_epochs=5000,
                warmup_iters=100, # how many steps to warm up for
                )


num decayed parameter tensors: 10, with 573,440 parameters
num non-decayed parameter tensors: 18, with 1,792 parameters
using fused AdamW: False


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
